In [25]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from copy import deepcopy
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc
import sys

In [29]:
# node class that each spot in the map will occupy
# cell location and goal_location are tuples representing index 
# of current cell location and goal cell locations
class Node:
    def __init__(self, parent, c2c, c2g, h, cell_location):
        self.parent = parent
        self.c2c = c2c
        self.c2g = c2g
        self.h = h
        self.cell_location = cell_location

In [30]:
# given 2 points of a line, retrun a lambda function which caluclates the 
# y value of an x
def generate_line_eq(p1, p2):
    
    x1 = p1[0]
    y1 = p1[1]
    
    x2 = p2[0]
    y2 = p2[1]
    
    m = (y2-y1)/(x2-x1)
    b = y1-m*x1
    
    lin_func = lambda x: m*x+b
    
    return lin_func

In [32]:
# create the board
# returns a 3d array
# dimensions are height width and angle. Takes in a compressed version of 
# the height width and angle which handles the region/node similarity
def create_board(compressed_width, compressed_height, compressed_angles):

    board = []
    for row_num in range(0, compressed_height):
        temp_row = []
        for col_num in range(0, compressed_width):

            temp_configuration = []
            for angle in compressed_angles:

                c2c = np.Infinity
                c2g = np.sqrt(row_num**2 + col_num**2 + angle**2)

                new_node = Node(parent=None, 
                                c2c=c2c,
                                c2g=c2g,
                                h=c2c+c2g,
                                cell_location=[row_num, col_num, angle])

                temp_configuration.append(new_node)
            temp_row.append(temp_configuration)
        board.append(temp_row)

    return board

In [6]:
# hardcoded obstacles defined by their vertices and origins
# we just see if the current x and y are within bounding lines
def check_obstacle(x, y):
    
    # check circle
    if y <= 225 and y >= 145 and x <= np.sqrt(40**2 - (y-185)**2) + 300 and x >= -np.sqrt(40**2 - (y-185)**2) + 300:
        return True
    
    # check triangles
    t1_line = generate_line_eq((36, 185), (115, 210))
    t2_line = generate_line_eq((36, 185), (102, 100))
    t3_line = generate_line_eq((80, 180), (115, 210))
    t4_line = generate_line_eq((80, 180), (105, 100))
    if x in range(36, 81):
        if y in range(int(np.floor(t2_line(int(x)))), int(np.ceil(t1_line(int(x))))+1):
            return True
    if x in range(80, 116):
        if y in range(int(np.floor(t3_line(int(x)))), int(np.ceil(t1_line(int(x))))+1):
            return True
    if x in range(80, 106):
        if y in range(int(np.floor(t2_line(int(x)))), int(np.ceil(t4_line(int(x))))+1):
            return True
    
    # check hexagon
    x_h = 200
    y_h = 100
    a = 35
    s = int(a*2/np.sqrt(3))
    
    h1_line = generate_line_eq((x_h-a, y_h+s/2), (x_h,y_h+s))
    h2_line = generate_line_eq((x_h-a, y_h-s/2), (x_h,y_h-s))
    h3_line = generate_line_eq((x_h,y_h+s), (x_h+a, y_h+s/2))
    h4_line = generate_line_eq((x_h,y_h-s), (x_h+a, y_h-s/2))
    
    if x in range(x_h-a, x_h+1):
        if y in range(int(np.floor(h2_line(int(x)))), int(np.ceil(h1_line(int(x))))+1):
            return True
    if x in range(x_h, x_h+a+1):
            if y in range(int(np.floor(h4_line(int(x)))), int(np.ceil(h3_line(int(x))))+1):
                return True

    return False




In [22]:
# iterate over the board, and if the cell is an obstacle, generate 
# the a circle of points around it which are padding
def generate_margin(color_map, radius):

    for y in range(len(color_map)):
        for x in range(len(color_map[y])):

            # read the color map and check to see if the current space is an obstacle
            if (color_map[y][x][0] == 255 and color_map[y][x][1] == 0 and 
                color_map[y][x][2] == 0):

                # generate circle bounds for a point if it is an obstacle
                x_range = range(x-radius, x+radius+1)

                for x_i in x_range:
                    y_upper_limit = np.ceil(np.sqrt(radius**2-(x_i-x)**2) + y)
                    y_lower_limit = np.floor(-np.sqrt(radius**2-(x_i-x)**2) + y)

                    y_range = np.arange(y_lower_limit, y_upper_limit+1)
                    for y_i in y_range:
                        if (x_i >= 0 and x_i <= 399) and (y_i >= 0 and y_i <= 249):
                            if not (color_map[int(y_i)][x_i][0] == [255] and 
                                    color_map[int(y_i)][x_i][1] == [0] and 
                                    color_map[int(y_i)][x_i][2] == [0]):
                                color_map[int(y_i)][x_i] = [255,255,0]
    return color_map


In [23]:
# read the board and depending on each nodes status
# write the proper color in a numpy array as BGR colors
def create_color_map(radius):

    color_map = np.zeros(shape=[250, 400, 3], dtype=np.uint8)

    for row in range(250):
        for col in range(400):
            if check_obstacle(col, row):
                color_map[row][col][0] = 255
                color_map[row][col][1] = 0
                color_map[row][col][2] = 0

            else:
                color_map[row][col][0] = 0
                color_map[row][col][1] = 0
                color_map[row][col][2] = 0


    color_map = generate_margin(color_map, radius)
    return color_map

In [ ]:
# will be used when iterating over closed nodes
# updates the previous color map given the current node to a specifies color
def update_color_map(curr_node, color_map,  brg_color):

    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]

    color_map[row][col][0] = brg_color[0]
    color_map[row][col][1] = brg_color[1]
    color_map[row][col][2] = brg_color[2]

    return color_map

In [7]:
def check_up(board, curr_node):
    
    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]
    
    # check out of bounds
    if row < 249:
        
        # check if obstacle
        new_node = board[row+1][col]
        
        if not new_node.is_margin:
            
            new_c2c = 1 + curr_node.c2c
            
            if new_c2c < new_node.c2c:
                new_node.c2c = new_c2c
                new_node.parent = curr_node
                
            return new_node

    return None

In [8]:
# generate next possible nodes for the current one
# filter out null ones which happens in a bpundary condition
def gen_next_nodes(board, curr_node):

    new_nodes = []

    new_nodes.append(check_up(board, curr_node))
    # new_nodes.append(check_down(board, curr_node))
    # new_nodes.append(check_left(board, curr_node))
    # new_nodes.append(check_right(board, curr_node))
    # new_nodes.append(check_up_left(board, curr_node))
    # new_nodes.append(check_up_right(board, curr_node))
    # new_nodes.append(check_down_left(board, curr_node))
    # new_nodes.append(check_down_right(board, curr_node))

    return list(filter(lambda node: node is not None, new_nodes))


In [9]:
# this is the backtracking function
# returns a list of nodes in order to find the solution
def get_solution_path(curr_node):
    solution_path= []
    
    while curr_node:
        solution_path.insert(0, curr_node)
        curr_node = curr_node.parent
        
    return solution_path

In [10]:
# use cv2 in order to draw how the node 
# traversal looks as well as plot the shortest path

def animate(color_map, closed_nodes, solution_path, filename):
    out = cv2.VideoWriter(f'{filename}.avi',cv2.VideoWriter_fourcc(*'DIVX'), 60, (400, 250))
 
    for node in closed_nodes:
        out.write(np.flipud(update_color_map(node, color_map, [255, 255, 255])))

    for node in solution_path:
        out.write(np.flipud(update_color_map(node, color_map, [0, 255, 0])))
        
    out.release()

In [ ]:
start_location, goal_location, margin = parse_input()

width = 400
height = 250

# create the starting board and color map based off of the starting board
print('Building Board')
board = create_board(width, height, margin)
color_map = create_color_map(board)

In [ ]:
if start_location[0] not in range(0, 250) or start_location[1] not in range(0, 400):
    print("Start Location Out Of Bounds")

if goal_location[0] not in range(0, 250) or goal_location[1] not in range(0, 400):
    print("Goal Location Out Of Bounds")

start_node = board[start_location[0]][start_location[1]]
goal_node = board[goal_location[0]][goal_location[1]]


if start_node.is_obstacle or start_node.is_margin:
    print('Cannot start in obstacle or obstacle margin')

if goal_node.is_obstacle or goal_node.is_margin:
    print('Cannot start in obstacle or obstacle margin')

In [ ]:
start_node.c2c = 0

open_nodes = [start_node]
closed_nodes = []

found = False

print('Searching')
while len(open_nodes) > 0:
    # generate the colors of the current board and append it to the list
    # this will be a frame of an animation
    # color_maps.append(gen_color_map(board)
    open_nodes.sort(key=lambda x: x.c2c)
    curr_node = open_nodes.pop(0)
    closed_nodes.append(curr_node)


    row = curr_node.cell_location[0]
    col = curr_node.cell_location[1]
    # print(f"Searching ({row},{col})")

    if curr_node.cell_location == goal_location:
        print('Found Solution')
        found = True
        break
    else:
        next_possible_nodes = gen_next_nodes(board, curr_node)
        for node in next_possible_nodes:

            appendable = True

            for o_node in open_nodes:
                if o_node == node:
                    appendable = False
                    break
            if appendable:
                for c_node in closed_nodes:
                    if c_node == node:
                        appendable = False
                        break

            if appendable:
                open_nodes.append(node)

if not found:
    print('No Solution')

In [ ]:
print('Animating Search Pattern')          
                    
# back track and animate the search and solution
solution_path = get_solution_path(curr_node)
animate(color_map, closed_nodes, solution_path, filename='search')
